نتیجه گیری:در لاندا های کوچک مدل اورفیت میکند
طوری که در درجات بالاتر و لاندای 10 بهترین نتیجه حاصل میشود

In [108]:
#import libraries
import numpy as np
import pandas as pd
from numpy.linalg import inv
import copy

In [109]:
#read files
trainData = pd.read_csv("train_set.csv")
testData = pd.read_csv("test_set.csv")

#give deep copy of them and then its all columns
trainCopy = copy.deepcopy(trainData)
for i in trainCopy.columns:
    trainCopy[i] = (trainCopy[i]-trainCopy[i].mean()) / trainCopy[i].std()
testCopy = copy.deepcopy(testData)
for i in testCopy.columns:
    testCopy[i] = (testCopy[i]-testCopy[i].mean()) / testCopy[i].std()

In [110]:
#constants definition
x_train = trainCopy.to_numpy()[:, 0:13]
y_train = trainCopy.to_numpy()[:, 13]
x_test = testCopy.to_numpy()[:, 0:13]
Landa = [0.1,1,10]
degrees = [1 , 3]
W = []
#pn represent power n list
p1=[]
p3=[]
p5=[]

In [111]:
#function definition block

def conversionVector(X , powerList):
    x = []
    for episode in X:
          temp = []
          for p in range(len(powerList)):
                s = 0
                for i in range(len(powerList[p])):
                    if powerList[p][i]!= 0 :
                        s += episode[i]**powerList[p][i]
                temp.append(s)
          x.append(temp)
    x = np.array(x)
    return x

#function for calculate polynomial regression on inputs
def polynomial_regerssion(xVec, yVec, dimention , powerList, Landa):
    X = []
    for episode in xVec:
          tX = []
          for p in range(len(powerList)):
                s = 0
                for i in range(len(powerList[p])):
                    if powerList[p][i]!= 0 :
                        s += episode[i]**powerList[p][i]
                tX.append(s)
          X.append(tX)
    X = np.array(X)
    
    return [((inv(np.transpose(X).dot(X) + Landa* np.identity(dimention))).dot(np.transpose(X))).dot(yVec) , copy.deepcopy(X)]

def MSE(W, X, Y):
    y = []    
    for i in range(len(Y)):
        predict = 0
        for j in range(len(W)):
            predict += W[j]*X[i][j]
        y.append(predict)
    sumE = 0
    for i in range(len(Y)):
        sumE += (Y[i] - y[i])**2
    return sumE / len(Y)

p1 = []
#this function get order and update powers list
def Power1(order, var, powers):
    for pow in range(order+1):
        powers[var-1] = pow
        if sum(powers) <= order:
            if tuple(powers) not in p1 :
                p1.append(tuple(powers))
        if var < 13:
            Power1(order,var+1,powers)
Power1(1 , 1 , [0]*13)
p1.reverse()
p1 = p1[:len(p1)-1]
     
         
p3 = []
#this function get order and update powers list
def Power3(order, var, powers):
    for pow in range(order+1):
        powers[var-1] = pow
        if sum(powers) <= order:
            if tuple(powers) not in p3 :
                p3.append(tuple(powers))
        if var < 13:
            Power3(order,var+1,powers)
            
Power3(3 , 1 , [0]*13)
p3.reverse()
p3 = p3[:len(p3)-1]

In [112]:


powerlists = [p1 , p3]    
updatedXTrain = x_train[0:len(x_train)-1]
updatedYTrain = y_train[0:len(x_train)-1]
W = []
#5 fold cross validation process 
for i in range(len(powerlists)):
    for l in Landa:
        for fold in range(5):
            firstIndex = fold*81
            result = polynomial_regerssion(np.concatenate((updatedXTrain[0:firstIndex], updatedXTrain[firstIndex+81:]), axis=0) , np.concatenate((updatedYTrain[0:firstIndex], updatedYTrain[firstIndex+81:]), axis=0) , len(powerlists[i]) , powerlists[i] , l)
            weight = result[0]
            W.append(weight)
            print("MSE Train set: ")
            print("degree:",degrees[i]," Landa:",l," Fold:",fold," MSE: ",MSE(weight, result[1] , np.concatenate((updatedYTrain[0:firstIndex], updatedYTrain[firstIndex+81:]), axis=0) ))
            print("MSE test set: ")
            print("degree:",degrees[i]," Landa:",l," Fold:",fold," MSE: ",MSE(weight, conversionVector(updatedXTrain[firstIndex:firstIndex+81] ,powerlists[i] ) , updatedYTrain[firstIndex:firstIndex+81]))
            print("")

greatestW = W[25]
finalPredict = []    

for i in range(len(x_test)):
    sumOfP = 0
    for j in range(len(greatestW)):
        sumOfP += greatestW[j]*(conversionVector(x_test ,powerlists[1] )[i][j])
    finalPredict.append(sumOfP)
print(finalPredict)

MSE Train set: 
degree: 1  Landa: 0.1  Fold: 0  :  0.24848202011323114
MSE test set: 
degree: 1  Landa: 0.1  Fold: 0  :  0.30906597119669194

MSE Train set: 
degree: 1  Landa: 0.1  Fold: 1  :  0.21302636275559717
MSE test set: 
degree: 1  Landa: 0.1  Fold: 1  :  0.4447719122610565

MSE Train set: 
degree: 1  Landa: 0.1  Fold: 2  :  0.2469051771843629
MSE test set: 
degree: 1  Landa: 0.1  Fold: 2  :  0.2863520894216315

MSE Train set: 
degree: 1  Landa: 0.1  Fold: 3  :  0.2694980994850224
MSE test set: 
degree: 1  Landa: 0.1  Fold: 3  :  0.18756743754238284

MSE Train set: 
degree: 1  Landa: 0.1  Fold: 4  :  0.25751395456508325
MSE test set: 
degree: 1  Landa: 0.1  Fold: 4  :  0.23385135338650975

MSE Train set: 
degree: 1  Landa: 1  Fold: 0  :  0.2485096475093099
MSE test set: 
degree: 1  Landa: 1  Fold: 0  :  0.30992569934102543

MSE Train set: 
degree: 1  Landa: 1  Fold: 1  :  0.21304922167352572
MSE test set: 
degree: 1  Landa: 1  Fold: 1  :  0.44524666288155534

MSE Train set: 
deg